In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, Conv2DTranspose, LeakyReLU, Flatten, Dense, Reshape, Input, Activation
from tensorflow.keras.datasets import mnist
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

from tensorflow.keras import backend as K

In [2]:
class ConvAutoencoder:
  def build(width, height, depth, filters=(32,64), latentDim=16):
    input_shape = (height, width, depth)
    chanDim = -1

    inputs = Input(shape=input_shape)
    x = inputs

    for filter in filters:
      x = Conv2D(filter, (3, 3), 2, padding='same')(x)
      x = LeakyReLU(0.2)(x)
      x = BatchNormalization(axis=chanDim)(x)

    volume_size = K.int_shape(x)
    x = Flatten()(x)
    latent = Dense(latentDim)(x)

    encoder = Model(inputs, latent, name='encoder')

    latent_inputs = Input(shape=(latentDim,))
    x = Dense(np.prod(volume_size[1:]))(latent_inputs)
    x = Reshape((volume_size[1], volume_size[2], volume_size[3]))(x)

    for filter in filters[::-1]:
      x = Conv2DTranspose(filter, (3, 3), 2, padding='same')(x)
      x = LeakyReLU(0.2)(x)
      x = BatchNormalization(axis=chanDim)(x)

    x = Conv2DTranspose(depth, (3, 3), padding='same')(x)
    outputs = Activation('sigmoid')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')

    return autoencoder

In [3]:
epoch = 20
batch_size = 32

((train_x, _), (test_x, _)) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
train_x = np.expand_dims(train_x, axis=-1)
test_x = np.expand_dims(test_x, axis=-1)

In [5]:
train_x = train_x.astype('float32')/255.0
test_x = test_x.astype('float32')/255.0

In [6]:
autoencoder = ConvAutoencoder.build(28, 28, 1)

In [7]:
autoencoder.compile(loss='mse', optimizer='adam')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 encoder (Functional)        (None, 16)                69392     
                                                                 
 decoder (Functional)        (None, 28, 28, 1)         109377    
                                                                 
Total params: 178769 (698.32 KB)
Trainable params: 178385 (696.82 KB)
Non-trainable params: 384 (1.50 KB)
_________________________________________________________________


In [9]:
history = autoencoder.fit(
    train_x, train_x,
    validation_data=(test_x, test_x),
    epochs = epoch,
    batch_size=batch_size
)

Epoch 1/20
1875/1875 [==============================] - 27s 6ms/step - loss: 0.0185 - val_loss: 0.0108
Epoch 2/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0104 - val_loss: 0.0093
Epoch 3/20
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0094 - val_loss: 0.0092
Epoch 4/20
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0087 - val_loss: 0.0083
Epoch 5/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0084 - val_loss: 0.0085
Epoch 6/20
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0081 - val_loss: 0.0080
Epoch 7/20
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0079 - val_loss: 0.0077
Epoch 8/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0077 - val_loss: 0.0077
Epoch 9/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0075 - val_loss: 0.0074
Epoch 10/20
1875/1875 [==============================] - 13s 7ms/step - l

In [11]:
import cv2
import numpy as np

decoded = autoencoder.predict(test_x)
outputs = None

for i in range(10):
  orig = (test_x[i] * 255).astype('uint8')
  constructed = (decoded[i] * 255).astype('uint8')

  output = np.hstack([orig, constructed])

  if outputs is None:
    outputs = output
  else:
    outputs = np.vstack((outputs, output))

# Save the result using cv2
cv2.imwrite('output_image.png', outputs)


313/313 [==============================] - 1s 2ms/step


True